## packge install
```

$ pip install oandapyV20
$ pip install backtesting==0.1.7
$ pip install TA-Lib
$ brew install ta-lib
$ conda install pandas
$ conda install configparser
$ conda install requests
```

In [11]:
import configparser
import datetime
import pandas as pd
from oandapyV20 import API

config = configparser.ConfigParser()
config.read("../oanda_config.txt")
api_key = config["Practice"]["api_key"]
oanda_api = API(api_key, "practice")

In [12]:
import oandapyV20.endpoints.instruments as instruments
def get_candles(instrument, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument, params)
    resp = oanda_api.request(r)
    data = []
    for candle in resp["candles"]:
        data.append([
            datetime.datetime.fromisoformat(candle["time"][:19]),
            candle["mid"]["o"],
            candle["mid"]["h"],
            candle["mid"]["l"],
            candle["mid"]["c"],
            candle["volume"],
        ])
    return data

In [13]:
UTC = datetime.timezone.utc
date_from = datetime.datetime(2020, 11, 3, 0, 0, tzinfo=UTC)
date_to = datetime.datetime(2020, 11, 9, 23, 59, tzinfo=UTC)
candles = get_candles("USD_JPY", date_from, date_to, "M5")

# DataFrame 型へ変換
df = pd.DataFrame(candles)
df.columns = ["Time", "Open", "High", "Low", "Close", "Volume"]
# カラムの型を float へ変換
df["Open"] = df["Open"].astype(float)
df["High"] = df["High"].astype(float)
df["Low"] = df["Low"].astype(float)
df["Close"] = df["Close"].astype(float)


print(df)
df.dtypes

                    Time     Open     High      Low    Close  Volume
0    2020-11-03 00:00:00  104.724  104.738  104.722  104.731     116
1    2020-11-03 00:05:00  104.730  104.741  104.730  104.732      72
2    2020-11-03 00:10:00  104.732  104.754  104.732  104.754      79
3    2020-11-03 00:15:00  104.752  104.773  104.752  104.758      99
4    2020-11-03 00:20:00  104.760  104.760  104.750  104.758      48
...                  ...      ...      ...      ...      ...     ...
1433 2020-11-09 23:35:00  105.190  105.190  105.138  105.164     374
1434 2020-11-09 23:40:00  105.166  105.191  105.146  105.180     400
1435 2020-11-09 23:45:00  105.181  105.184  105.082  105.092     528
1436 2020-11-09 23:50:00  105.093  105.133  105.089  105.130     522
1437 2020-11-09 23:55:00  105.132  105.146  105.108  105.138     365

[1438 rows x 6 columns]


Time      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [14]:
from backtesting import Strategy
from backtesting.lib import crossover
import talib

def EMA_Backtesting(values, n):
    """
    Return exponential moving average of `values`, at
    each step taking into account `n` previous values.
    """
    close = pd.Series(values)
    return talib.EMA(close, timeperiod=n)

class EmaCrossStrategy(Strategy):

    # Define the two EMA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 10

    def init(self):
        # Precompute two moving averages
        self.ema1 = self.I(EMA_Backtesting, self.data.Close, self.n1)
        self.ema2 = self.I(EMA_Backtesting, self.data.Close, self.n2)

    def next(self):
        # If ema1 crosses above ema2, buy the asset
        if crossover(self.ema1, self.ema2):
            self.position.close()
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.ema2, self.ema1):
            self.position.close()
            self.sell()

In [15]:
from backtesting import Backtest

df_ = df.copy()
bt_ema = Backtest(df_, EmaCrossStrategy, cash=1, commission=0, trade_on_close=True)
bt_ema.run()
bt_ema.plot()


<ipython-input-15-1b53b2ac4094>:4: UserWarning: Data index is not datetime. Assuming simple periods.
  bt_ema = Backtest(df_, EmaCrossStrategy, cash=1, commission=0, trade_on_close=True)
